# 2nd part - Get the GPS coordinate for each postal code

To this end, I will use pgeocode. It has the major advantage to be an offline database. The drawback of this library is the postcode management for Canada.

In [1]:
#library to get the GPS coordinates offline (like, in the train)
#!pip install pgeocode
import pgeocode
# select the country
nomi = pgeocode.Nominatim('CA')
# the postal code needs to be entered like XXX YYY (issue marked on Git)
# where XXX is discarded, example:
nomi.query_postal_code('XXX M3A')

postal_code                                                     M3A
country code                                                     CA
place_name        North York (York Heights / Victoria Village / ...
state_name                                                  Ontario
state_code                                                       ON
county_name                                             North York 
county_code                                                     NaN
community_name                                                  NaN
community_code                                                  NaN
latitude                                                    43.7545
longitude                                                    -79.33
accuracy                                                          1
Name: 0, dtype: object

As the assignment ask for 3 distinct notebooks... Here is the code to get the dataframe, done in the previous part. No explanation will be given here:

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source, 'lxml')
data=[]
header=[]
table= soup.find('table',attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    head = row.find_all('th')
    cols = [ele.text.strip() for ele in cols]
    heads= [ele.text.strip() for ele in head]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
    header.append([ele for ele in heads if ele])
del(data[0]) # supprimer la ligne de départ
header= list(filter(None,header))
df = pd.DataFrame(data, columns=header[0])
deleted='Not assigned'
for row in df.itertuples():    
    if row.Neighbourhood == deleted:
        df.Neighbourhood.iloc[row.Index] = df.Borough.iloc[row.Index]
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
print('There is',df.shape[0], 'rows and',df.shape[1],'columns in the dataframe.')

There is 103 rows and 3 columns in the dataframe.


Here we go, the dataframe df contains the Postcode, Borough and Neighbourhood.

# A. Get the longitude and latitude information

In [3]:
# add the 2 columns latitude and longitude to df
df['latitude']=""
df['longitude']=""
# select the country
nomi = pgeocode.Nominatim('CA')

# get the info for each postalcode:
for lieu in df.itertuples():
    pcode =  'XXX ' + lieu.Postcode
    location = nomi.query_postal_code(pcode)
    df.ix[lieu.Index,'longitude']=location.longitude
    df.ix[lieu.Index, 'latitude']=location.latitude
    
df

C:\Users\Yann\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Yann\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8113,-79.193
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
5,M1J,Scarborough,Scarborough Village,43.7464,-79.2323
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.7298,-79.2639
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.7122,-79.2843
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.7247,-79.2312
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.6952,-79.2646


Surprisingly, the coordinates are slightly different than in the provided csv.

See you on the next file!